In the previous notebook, we looked extensively at designing non-aromatic molecules in RetroTide using a custom set of starters and extenders. We will perform a similar analysis in this notebook but by expanding our starter and extender set to attempt the biosynthesis of more complex, aromatic molecules

In [1]:
import bcs
from typing import Optional, List
from rdkit import Chem
from rdkit.Chem import AllChem

ModuleNotFoundError: No module named 'bcs'

In [ ]:
def modify_bcs_starters_extenders(starter_codes: Optional[List[str]] = None, 
                                  extender_codes: Optional[List[str]] = None):
    
    for key in list(bcs.starters.keys()):
        if key not in starter_codes:
            bcs.starters.pop(key, None) # removes key corresponding to the starter code that was not specified
            
    for key in list(bcs.extenders.keys()):
        if key not in extender_codes:
            bcs.extenders.pop(key, None) # same as above but for extenders